In [1]:
import requests
from datetime import datetime, timedelta
import traceback
import time
import json
import sys
import pandas as pd
import random
import numpy as np

# https://github.com/Watchful1/Sketchpad/blob/master/postDownloader.py

In [ ]:
results = []
i = 0
fields = ["author","author_fullname","created_utc","domain","is_self","title","selftext"]

start = datetime(2019,1,1)

while start <= datetime(2020,12,31):

    time.sleep(round(random.random()*5,2))
    
    end = start + timedelta(days=1)
    print(start,end)

    start_s = int(start.timestamp())
    end_s = int(end.timestamp())

    url = f"https://api.pushshift.io/reddit/search/submission/?subreddit=depression&after={start_s}&before={end_s}&size=100"
    r = requests.get(url)
    data = json.loads(r.text)["data"]
    for d in data:
        record = {}
        for f in fields:
            try:
                record[f] = d[f]
            except:
                record[f] = None 
        results.append(record)

    start = end
    i+=1

,author,year-month,is_self,subreddit,subreddit_subscribers,title,selftext,post_length,depressed
0,--liveitup,2019-01,True,hacking,647687,Hulu account hacked. Genuinely interested in h...,"Secure wifi, only use Hulu through a Roku stic...",183,1
1,-RL-Lokei,2019-01,True,starcitizen,148021,Yela jumptown/ druglab location,"Hi all,\n\nBefore you refer me onto videos of ...",64,1
2,-aegon-,2019-01,True,Judaism,29423,Advice for a poor convert-to-be?,How can I tell my religious Christian family t...,44,1
3,0haltja16,2019-01,True,confession,1149851,I cheated on a 5 year old’s AR tests,In grade school the older kids were forced to ...,115,1
4,101kbye,2019-01,True,AskWomenOver30,31874,Has anyone taken a sabbatical and what was rea...,I’m not burnt out at work but I want a 3 month...,174,1
...,...,...,...,...,...,...,...,...,...
637,teymourbeydoun,2020-12,True,Inception,7370,Am I missing Behind-the-Scenes footage on Blu-...,"Hello, I’ve just bought the 4K UHD Christopher...",69,0
638,HoopsFella,2020-12,True,NBAtradeideas,936,Surprise Harden Destinations,Curious where people think Harden could end up...,123,0
639,friedshrimpt,2020-12,True,Pratt,490,Is Pratt requiring ACT scores?,My score is very low (23). My gpa is a weighte...,31,0
640,zorbsthegreat,2020-12,True,MRKH,288,Tracking Hormonal Cycles and mood shifts with ...,"I have been diagnoses since 17 and Im 23 now, ...",194,0


In [12]:
# pd.DataFrame(results).to_csv(f"depression_users.psv", sep="|")
pd.DataFrame(results).to_csv(f"dep_users.csv")

In [2]:
dep_users = pd.read_csv("dep_users.csv")

In [3]:
dep_users['author'].nunique()

51828

In [6]:
author_counts = dep_users['author'].value_counts().reset_index().sort_values(by="author", ascending=False)
author_counts.columns = ["author","count"]
np.quantile(author_counts["count"], q=[0.9, 0.95, 0.99])

array([2., 3., 7.])

In [7]:
author_counts = author_counts.loc[author_counts["count"] <= 3]

In [8]:
author_counts['author'].nunique()

49849

In [35]:
dep_users_filt = dep_users.merge(author_counts, how="inner", on="author")
dep_users_filt = dep_users_filt.loc[dep_users_filt["selftext"] !="[removed]"]
dep_users_filt = dep_users_filt[["author","author_fullname","created_utc", "domain", "is_self", "title", "selftext","count"]]

dep_users_filte

In [45]:
dep_users_filt["year"] = dep_users_filt["created_utc"].apply(lambda x: datetime.fromtimestamp(x).year)
dep_users_filt["month"] = dep_users_filt["created_utc"].apply(lambda x: datetime.fromtimestamp(x).month)
dep_users_filt["year-month"] = dep_users_filt.apply(lambda x: f"{x['year']}-{str(x['month']).zfill(2)}", axis=1)
dep_users_single = dep_users_filt.groupby("author").first().reset_index()

In [46]:
dep_users_single.to_csv("dep_users_cohort.csv")